**Group Members** : 

--- 
1) Name : Garima Dewangan  
Roll no. : 19D070023  

---  
2) Name : Kimaya Shikarkhane  
Roll no. : 19D070053 

---
3) Name : Mitali Singal  
Roll no. : 19D070036

---

##Input

**Format** :  
Netlist.txt
'\*' symbol is for comments. Comments are ignored.  
empty lines are ignored  
1st line is the inputs list with space between two inputs  
2nd line is the list of outputs with space between two outputs  
From the 3rd line the gates are described  
**format of describing gate** :   
*output gate inputs*  
variable names are cases sensitive  


In [ ]:
#Opening netlist file
st = input("Netlist file : ")
f1 = open(st, "r")

Netlist file : netlist1.txt


##Code

In [ ]:
import sys

#Gate definitions
def AND(a):
  #print(a)
  if '0' in a:
    return('0')
  elif 'x' in a:
    #print('x')
    return('x')
  else:
    return('1')

def OR(a):
	if '1' in a:
		return('1')
	elif 'x' in a:
		return('x')
	else:
		return('0')

def NOT(a):
	if(a[0]=='1'):
		return('0')
	elif(a[0]=='0'):
		return('1')
	else:
		return('x')

def XOR(a):
	if 'x' in a:
		return('x')
	elif(a.count('1')%2==0 or a.count('0')%2==0):
		return('0')
	else:
		return('1')

def NOR(a):
	return(NOT(OR(a)))

def NAND(a):
	return(NOT(AND(a)))

In [ ]:
#Node of the graph
class node:
	def __init__(self,type,inp):
		self.type = type
		self.inp = inp

In [ ]:
#Function to perform calculations
def calc(type,inps):
  #print(type,inps)
  if type == 'and':
    return(AND(inps))
  elif type == 'or':
    return(OR(inps))
  elif type == 'not':
    return(NOT(inps))
  elif type == 'xor':
    return(XOR(inps))
  elif type == 'nor':
    return(NOR(inps))
  elif type == 'nand':
    return(NAND(inps))
  else:
    return('x')

In [ ]:
#Evaluates output
def evaluate(x):
  #print(x.type,x.inp)
  inps = []
  for i in x.inp:
    #print(x.type,i)
    if i not in inputs:
      if(variable[i]=='x'):
        variable[i] = evaluate(graph[i])
      #print(i,variable[i])
    inps.append(variable[i]) 
  val = calc(x.type,inps)
  #print(x.type,inps,val)
  return (val)

In [ ]:
#Fault sensitization/ Initializes the variable with a given value - by initializing th inputs
def initialize(a,v):
  x = graph[a]
  if (x.type == 'not' or x.type == 'nor' or x.type == 'nand' ):
    if v == '1':
      r = '0'
    else: 
      r = '1'
  else:
    r = v
  #print('start to init',a,r,x.type,x.inp)
  for i in x.inp:
    #print(i)
    if variable[i] == 'x':
      if i in inputs:
        variable[i] = r
      else:
        initialize(i,r)
    inps = []
    for iv in x.inp:
      inps.append(variable[iv])
    val = calc(x.type,inps)
    #print('calc-mid',a,x.type,'inputs',x.inp,variable[x.inp[0]],variable[x.inp[1]],'value',val,'return')
    if val == r:
      break
    else:
      continue
  variable[a] = val
  #print('Intialized to',variable[a],'using',x.inp,variable[x.inp[0]],variable[x.inp[1]],'return')
  return 

In [ ]:
#Assigns non-controlling value according to gate
def assign(type):
  if type == 'and' or type == 'nand' :
    return ('1')
  else:
    return ('0')

In [ ]:
#Finds a path from the faulty node to the output
def path(node1, node2):
  path = []
  visited = {}
  for i in variable.keys():
    visited[i] = 0
  n = node2
  path.append(n)
  visited[n] = 1
  while 0 in visited.values():
    for i in graph[n].inp:
      if (visited[i]==1):
        continue
      else:
        visited[i]=1
        path.append(i)
        if (i==node1):
          return (path)
        else:
          if i in inputs:
            path.pop()
          else:
            n = i
    if len(path)>0:
      n = path.pop()
    else:
      break
  return path

In [ ]:
#Generates test vector for a given fault at a given node
def generate(f,node):
  #print('initialize')
  #If fault is SA)
  if f=='SA0':
    #Initialize that node to 1
    if node in inputs:
      variable[node] = '1'
    else:
      initialize(node,'1')
  #If fault is SA!
  else:
    #Initialize the node to 0
    if node in inputs:
      variable[node] = '0'
    else:
      initialize(node,'0')
  initial = {}
  for i in variable.keys():
    initial[i] = variable[i]
  #print(initial)
  #Propogating the fault to output by assigning non controoling values to gates along the path 
  for o in outputs:
    for i in initial.keys():
      variable[i] = initial[i]
    xo = graph[o]
    #print('evaluate')
    variable[o] = evaluate(graph[o])
    #print('output',o,variable[o])
    if variable[o] == 'x':
      #->
      path0 = path(node,o)
      #print('path',path0)
      if(len(path0)==0):
        continue
      else :
        n = len(path0)-2
        while (n>=0):
          x = path0[n]
          while (variable[x]=='x'):
            for i in graph[x].inp:
              #print(i)
              if variable[i]=='x':
                if i in inputs:
                  variable[i] = assign(graph[x].type)
                else:
                  #print('init',i,assign(graph[x].type))
                  initialize(i,assign(graph[x].type))
              variable[x] = evaluate(graph[x])
              if variable[x]!='x':
                break
            #print('While-loop',x,variable[x],graph[x].inp)
          #print(path0[0])
          evaluate(graph[path0[0]])
          n = n-1
      if variable[o] != 'x':
        break
  ans = []
  for i in inputs:
    ans.append(variable[i])
  return ans

In [ ]:
#Generates vectors for all stuck at faults for all nodes
def ATPG(variable,graph,inputs,outputs):
  test_vec = []
  #print(variable.keys())
  var = []
  for k in variable.keys():
    var.append(k)
  for v in var:
    print(v)
    for i in variable.keys():
      variable[i] = 'x'
    #print(variable)
    vec = generate('SA0',v)
    print('SA0',vec)
    #print(variable)
    if vec not in test_vec:
      test_vec.append(vec)
    for i in variable.keys():
      variable[i] = 'x'
    vec = generate('SA1',v)
    print('SA1',vec)
    #print(variable)
    if vec in test_vec:
      test_vec.append(vec)
  return(test_vec)

In [ ]:
#Declaring dictionaries to store variable values and nodes of graph
variable = {}
graph = {}

t =0
for x in f1:
	x = x.strip()
	if len(x)==0 or x[0]=='*' :
		continue
	elif t==0:
		inputs = x.split()
		for i in range(len(inputs)):
			variable[inputs[i]] = 'x'
		#print(variable)
		t=t+1
	elif t==1:
		outputs = x.split()
		t = t+1
	else:
		code = x.split()
		variable[code[0]] = 'x'
		#print(code[0],code[1],code[2:])
		x = node(str(code[1].lower()),code[2:])
		graph[code[0]] = x
		#print(x.type)


##Output

In [ ]:
test_vec = ATPG(variable,graph,inputs,outputs)

a
SA0 ['1', '0', '0']
SA1 ['0', '0', '0']
b
SA0 ['0', '1', '0']
SA1 ['0', '0', '0']
cin
SA0 ['0', '0', '1']
SA1 ['0', '0', '0']
1
SA0 ['1', '1', '0']
SA1 ['0', '0', '0']
s
SA0 ['1', '1', '1']
SA1 ['0', '0', '0']
2
SA0 ['1', '1', '1']
SA1 ['0', 'x', '0']
3
SA0 ['1', '1', 'x']
SA1 ['0', '1', '1']
cout
SA0 ['1', '1', '1']
SA1 ['0', 'x', '0']


In [ ]:
print(test_vec)

[['1', '0', '0'], ['0', '1', '0'], ['0', '0', '1'], ['1', '1', '0'], ['1', '1', '1'], ['1', '1', 'x']]
